In [ ]:
import matplotlib.pyplot as plt
import cv2, os
import tensorflow as tf
import numpy as np

In [ ]:
DATADIR = "/path/to/dataset/"
CATEGORIES = ["no", "yes"]

imageSize = 255

In [ ]:
trainingData = []
def DataPreprocessig():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        classNum = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                rawImage = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resizeImage = cv2.resize(rawImage, (imageSize, imageSize))
                trainingData.append([resizeImage, classNum])
            except Exception as e:
                pass
            
DataPreprocessig()
print(len(trainingData))

In [ ]:
F = []
l = []

for features, labels in trainingData:
    F.append(features)
    l.append(labels)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    F, l, test_size = 0.2, random_state = 42, shuffle = True)

x_train = np.array(x_train)
x_train = x_train.reshape(-1, 255,255,1)
y_train = np.array(y_train)

x_test = np.array(x_train)
x_test = x_train.reshape(-1, 255,255,1)
y_test = np.array(y_train)

In [ ]:
from tensorflow.keras import layers
dataAugmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("horizontal",
        input_shape=(255,255,1)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [ ]:
import tensorflow.keras.layers
checkModel = tf.keras.models.Sequential([
    dataAugmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Conv2D(256, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Conv2D(512, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(2048, activation='relu'),
    layers.Dense(2, name="outputs", activation = 'softmax')
])

In [ ]:
checkModel.compile(
  optimizer = 'Adam',
  loss = tf.keras.losses.SparseCategoricalCrossentropy (from_logits = True),
  metrics = ['accuracy'])

checkModelHistory = checkModel.fit(x_train, y_train, batch_size = 21, epochs = 20, 
                                 validation_data = (x_test, y_test))

In [ ]:
checkModel.save("model.h5")

In [ ]:
def modelPlotting():
    plt.title('model accuracy')
    plt.plot(checkModelHistory.history['accuracy'])
    plt.plot(checkModelHistory.history['val_accuracy'])
    plt.ylabel('Accuracy')
    plt.xlabel('Epoches')
    plt.legend(['Train', 'Val'], loc = 'lower right')
    plt.show()
    
    plt.title('model loss')
    plt.plot(checkModelHistory.history['loss'])
    plt.plot(checkModelHistory.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('Epoches')
    plt.legend(['Train', 'Val'], loc = 'upper right')
    plt.show()
    
modelPlotting()

In [ ]:
test_loss, test_acc = checkModel.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')